# LinkedIn Market Analysis


### Process:
1. Scrape data from Linkedin and Glassdoor, using Selenium.
2. EDA, cleaning, and export to CSV.
3. Compare to previous years' data (2 and 4 years ago) using tableau.

### Additional Datasets:
- 2018: 
- 2020: 
- 2022: [my submission to Kaggle]

### Reference Notebooks:
- https://www.kaggle.com/code/gawainlai/us-data-science-job-salary-regression-w-visuals (beyond my skill level)
- https://www.kaggle.com/code/discdiver/the-most-in-demand-skills-for-data-scientists (top skills)

## Import Libraries

In [1]:
# selenium imports
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.action_chains import ActionChains

# web scraping imports
from bs4 import BeautifulSoup
import requests

# database imports
import re as re
import time
import pandas as pd
import os
from pymongo import MongoClient
import numpy as np

# import and load file to login to LinkedIn
from dotenv import load_dotenv
load_dotenv()

True

## Scrape LinkedIn Using the Selenium Driver

In [2]:
# initialize the LinkedIn scrape

# Options
options = webdriver.ChromeOptions() # init for chrome
options.add_argument('--incognito') # runs chrome in a 'clean slate' window
#options.add_argument('--headless') # runs chromedriver in the background, without opening a window

# Initialize the selenium driver
driver = webdriver.Chrome(options = options, executable_path='./chromedriver')
login_url = "https://www.linkedin.com/uas/login"

# Start the page
driver.get(login_url)
time.sleep(3)

# Target the login elements
email = driver.find_element("id", "username")
password = driver.find_element("id", "password")

# Load env variables
my_email = os.getenv("linkedin_username")
my_password = os.getenv("linkedin_password")

# Input in the form
email.send_keys(my_email)
password.send_keys(my_password)
password.send_keys(Keys.RETURN)

/var/folders/yx/4xsyrpbs305132t_87mnd3600000gn/T/ipykernel_74274/3666220930.py:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(options = options, executable_path='./chromedriver')


In [3]:
"""def scrape(data_role):
    """""" Scrape 40 pages of LinkedIn job search, for the given data role """"""
    
    # create lists to store all scraped data (12 criteria)
    search, titles, companies, locations, remote, post_dates, num_applicants, full_time, size, easy_apply, desc, salaries = [], [], [], [], [], [], [], [], [], [], [], []
        
    for i in range(2): # FIX: change to 40. scrape jobs until page=40
        print(i+1, 'of 2 pages.')
        
        # navigate to the correct page
        scrape_url = f"https://www.linkedin.com/jobs/search/?&keywords={data_role}&refresh=true&start={i*25}"
        if i == 0:
            scrape_url = f"https://www.linkedin.com/jobs/search/?&keywords={data_role}&refresh=true&start={1}"
        driver.get(scrape_url)
        time.sleep(5)

        # SCROLL
        
        
        time.sleep(10)
        """"""
        # increment to next page
        #driver.find_element("xpath", f'//button[@aria-label="Page {i}"]').click()
        if i < 9:
            driver.find_element(by=By.XPATH, value=
                f'/html/body/div[6]/div[3]/div[4]/div/div/main/div/section[1]/div/div[6]/ul/li[{i+1}]/button').click()
                  /html/body/div[6]/div[3]/div[4]/div/div/main/div/section[1]/div/div[6]/ul/li[1]/button
        elif i < 33:
            driver.find_element("xpath", 
                '/html/body/div[6]/div[3]/div[4]/div/div/main/div/section[1]/div/div[6]/ul/li[7]/button').click()
        else:
            driver.find_element("xpath", 
                f'/html/body/div[6]/div[3]/div[4]/div/div/main/div/section[1]/div/div[6]/ul/li[{i-28}]/button').click()
        """"""
        
        # convert page text to beautiful soup
        src = driver.page_source
        soup_for_page = BeautifulSoup(src, 'lxml')
        
        # create a list of jobs on the current page, to iterate through after each scrape
        job_list = []
        jobs_on_page = soup_for_page.find_all("a", attrs={"class":"disabled ember-view job-card-container__link job-card-list__title"})
        for i in jobs_on_page: # length of jobs varies by page
            job_list.append(i["href"])
            print(i["href"])
        print('length of job list: ', len(job_list)) # FIX: debugging
        # print(job_list) # FIX: debugging
            
        # scrape all jobs on the current page
        for i in range(len(job_list)):
            #print('index of job list while looping: ', i)
            
            
            WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "a[href=/jobs/view/3180134475/?eBP=CwEAAAGC1WAJO4bYPFukittYtSpBObZh0R9VanwII8SRPftbBrtLOH6kpCOuTagUlr1pu3vL9UUPR6WM3Sk8gmGLXHpoQ9527cecGaIauJjkEx3mw5ROl9WABoi-WDn2IYryS0W_kfex8vmXNwsq2VdHxj1TSEnidM_OLEE0Z_Almjae3-7MW2R9e-y9_ZsPvW5tnCWfoqq9Eq7kTsgVggwGelIvII2CMn0zs9EV7dkLFdfRekr7II9igjjFrsikEHD9RKZLVHuBc6GTjEnZVkT2nPbt_cns80xGxSyUt1Ozk1glHLeLMC6xe-HuQIALB2t-moR3cRdCGHGX5J3UF-YW&recommendedFlavor=ACTIVELY_HIRING_COMPANY&refId=8ZHQ5KpI5pIlGSwHIyNoyg%3D%3D&trackingId=gnVgCfpSBh21IA2z06Cyaw%3D%3D&trk=flagship3_search_srp_jobs/]"))).click()
            #driver.find_element(By.CSS_SELECTOR, "disabled ember-view job-card-container__link job-card-list__title").click()
            # ALL ATTEMPS TO CLICK ON A JOB WITH SELENIUM: use selenium to click on next job
            #driver.find_element(job_list[i]).click() #also try elements
            #driver.find_element(By.linkText(job_list[0])).Click();
            #link = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.class, '"disabled ember-view job-card-container__link job-card-list__title"')))
            #ActionChains(driver).move_to_element(job_list[0]).click().perform()
            #driver.find_element(By.)
            #driver.findElement(By.xpath("//a[@href=\"" + job_list[0] + "\"]")).Click() #for xpath
            #driver.find_element_by_css_selector(f'[href^=http://linkedin.com/{job_list[0]}]').Click() # for links
            #driver.find_element(By.CSS_SELECTOR, f'[href^={job_list[0]}]').Click() # for links
            #driver.find_element(By.XPATH, f'/html/body/div[5]/div[3]/div[3]/div[2]/div/section[1]/div/div/ul/li[{2}]/div/div/div[1]/div[2]/div[1]/a').click()
            

            #time.sleep(5)
            
            # convert page text to beautiful soup
            src = driver.page_source
            soup = BeautifulSoup(src, 'lxml') # FIX: rename soup_for_job and replace all var names
            
            
            
            # Return data of results on current selected sub-page to new lists
            #time.sleep(5)
            titles.append(soup.select('h2.t-24.t-bold.jobs-unified-top-card__job-title')[0].get_text().replace('\n','').strip())
            companies.append(soup.select('span.jobs-unified-top-card__company-name')[0].get_text().replace('\n','').strip())
            locations.append(soup.select('span.jobs-unified-top-card__bullet')[1].get_text().replace('\n','').strip())
            remote.append(soup.select('span.jobs-unified-top-card__workplace-type')[0].get_text().replace('\n','').strip())
            
            try:
                post_dates.append(soup.select('span.jobs-unified-top-card__posted-date')[0].get_text().replace('\n','').strip())
            except:
                print("could not find 'posted date'")
                post_dates.append(None)
            
            try:
                num_applicants.append(soup.select('span.jobs-unified-top-card__bullet')[2].get_text().replace('\n','').strip())
            except:
                print("could not find 'number of applicants'")
                num_applicants.append(None)
            
            try:
                full_time.append(soup.select('li.jobs-unified-top-card__job-insight')[0].get_text()
                    .replace('\n','').strip().rsplit(' ', 1)[-1])
            except:
                print("could not find 'full time'")
                full_time.append(None)
            
            #time.sleep(5)
            
            try:
                print(soup.select('li.jobs-unified-top-card__job-insight')[3].get_text())
                size.append(soup.select('li.jobs-unified-top-card__job-insight')[3].get_text().replace('\n','').strip())
            except:
                print("could not find 'company size'")
                size.append(None)
            
            try:
                print(soup.select('span.artdeco-button__text')[42].get_text())
                easy_apply.append(soup.select('span.artdeco-button__text')[42].get_text())
                #alt: div.jobs-apply-button--top-card
            except:
                print("could not find 'easy apply button'")
                easy_apply.append(None)
            
            desc.append(soup.select('div.jobs-box__html-content.jobs-description-content__text.t-14.t-normal.jobs-description-content__text--stretch')[0].get_text().replace('\n',' ').strip())
            
            try:
                salaries.append(soup.select('p.t-16')[0].get_text().replace('\n','').strip())
            except:
                print("could not find 'salary' on page")
                try:
                    salaries.append(re.find('($.)', desc).replace('\n','').strip())
                except:
                    print("could not find 'salary' in description")
                    try: 
                        salaries.append(soup.select('li.jobs-unified-top-card__job-insight')[0].get_text()
                            .replace('\n','').strip().rsplit(' ', 1)[0].rstrip(' ·'))
                    except:
                        print("could not find 'salary' in full-time")
                        salaries.append(None)
            
            search.append(data_role.replace('%20',' '))
    
    
    # pass data from lists into a dictionary
    dict_from_scrape = {'search':search, 'title':titles, 'company':companies, 'location':locations, 'post_date':post_dates, 
                        'num_applicants':num_applicants, 'full_time':full_time, 'size':size, 
                        'easy_apply':easy_apply, 'description':desc, 'salary':salaries}
    
    df_from_scrape = pd.DataFrame(dict_from_scrape)
    
    return df_from_scrape
"""

'def scrape(data_role):\n     Scrape 40 pages of LinkedIn job search, for the given data role \n    \n    # create lists to store all scraped data (12 criteria)\n    search, titles, companies, locations, remote, post_dates, num_applicants, full_time, size, easy_apply, desc, salaries = [], [], [], [], [], [], [], [], [], [], [], []\n        \n    for i in range(2): # FIX: change to 40. scrape jobs until page=40\n        print(i+1, \'of 2 pages.\')\n        \n        # navigate to the correct page\n        scrape_url = f"https://www.linkedin.com/jobs/search/?&keywords={data_role}&refresh=true&start={i*25}"\n        if i == 0:\n            scrape_url = f"https://www.linkedin.com/jobs/search/?&keywords={data_role}&refresh=true&start={1}"\n        driver.get(scrape_url)\n        time.sleep(5)\n\n        # SCROLL\n        \n        \n        time.sleep(10)\n        \n        # increment to next page\n        #driver.find_element("xpath", f\'//button[@aria-label="Page {i}"]\').click()\n      

In [4]:
def scrape_links(data_role):
    """ Scrape 40 pages of a LinkedIn job search for job links, using the given data role as the search term """
    
    # SCRAPE 40 PAGES
    for i in range(2): # FIX: set 2 to 40 for final product
        print(f'running search for {i+1} of 40 pages for {data_role}.')
        
        # navigate to the correct page
        scrape_url = f"https://www.linkedin.com/jobs/search/?&keywords={data_role}&refresh=true&start={i*25}"
        if i == 0:
            scrape_url = f"https://www.linkedin.com/jobs/search/?&keywords={data_role}&refresh=true&start={1}"
        driver.get(scrape_url)
        time.sleep(5)

        # SCROLL DOWN THE JOB LIST
        #jobs_block = driver.find_element_by_class_name('jobs-search-results__list')
        #jobs_list= jobs_block.find_elements(By.CSS_SELECTOR, '.jobs-search-results__list-item')
        # scroll down for each job element
        #driver.execute_script("arguments[0].scrollIntoView();", job)
        #time.sleep(10)
        
        # convert page text to beautiful soup
        src = driver.page_source
        soup_for_page = BeautifulSoup(src, 'lxml')
        
        # create a list of jobs on the current page, to iterate through after each scrape
        job_links = []
        jobs_on_page = soup_for_page.find_all("a", attrs={"class":"disabled ember-view job-card-container__link job-card-list__title"})
        for i in jobs_on_page: # length of jobs varies by page
            job_links.append(i["href"])
        print('length of job links: ', len(job_links)) # FIX: debugging
        # print(job_list) # FIX: debugging
        
    return job_links[0]

In [7]:
def scrape_listing(links):
    """ Returns all scraped data for each job listing from the links passed into the function """

    # VARIABLE ASSIGNMENT create lists to store all scraped data (12 criteria)
    search, titles, companies, locations, remote, post_dates, num_applicants, full_time, size, easy_apply, desc, salaries = [], [], [], [], [], [], [], [], [], [], [], []
    
    # scrape all jobs on the current page
    for key, value in job_links.items():
        
        # Navigate to page
        print('\n\n\nkey:', key, '\nvalue:', value[0][0])
        driver.get(f'https://linkedin.com{value[0][0]}')
        time.sleep(5)
        
        #click see more
        #scroll down??
        #time.sleep(5)
        
        # convert page text to beautiful soup
        src = driver.page_source
        soup = BeautifulSoup(src, 'lxml')
        
        # Return data of results on current selected sub-page to new lists
        titles.append(soup.select('h2.t-24.t-bold.jobs-unified-top-card__job-title')[0].get_text().replace('\n','').strip())
        companies.append(soup.select('span.jobs-unified-top-card__company-name')[0].get_text().replace('\n','').strip())
        locations.append(soup.select('span.jobs-unified-top-card__bullet')[1].get_text().replace('\n','').strip())
        remote.append(soup.select('span.jobs-unified-top-card__workplace-type')[0].get_text().replace('\n','').strip())

        try:
            post_dates.append(soup.select('span.jobs-unified-top-card__posted-date')[0].get_text().replace('\n','').strip())
        except:
            print("could not find 'posted date'")
            post_dates.append(None)

        try:
            num_applicants.append(soup.select('span.jobs-unified-top-card__bullet')[2].get_text().replace('\n','').strip())
        except:
            print("could not find 'number of applicants'")
            num_applicants.append(None)

        try:
            full_time.append(soup.select('li.jobs-unified-top-card__job-insight')[0].get_text()
                .replace('\n','').strip().rsplit(' ', 1)[-1])
        except:
            print("could not find 'full time'")
            full_time.append(None)

        try:
            print(soup.select('li.jobs-unified-top-card__job-insight')[3].get_text())
            size.append(soup.select('li.jobs-unified-top-card__job-insight')[3].get_text().replace('\n','').strip())
        except:
            print("could not find 'company size'")
            size.append(None)

        try:
            print(soup.select('span.artdeco-button__text')[42].get_text())
            easy_apply.append(soup.select('span.artdeco-button__text')[42].get_text())
            #alt: div.jobs-apply-button--top-card
        except:
            print("could not find 'easy apply button'")
            easy_apply.append(None)

        desc.append(soup.select('div.jobs-box__html-content.jobs-description-content__text.t-14.t-normal.jobs-description-content__text--stretch')[0].get_text().replace('\n',' ').strip())

        try:
            salaries.append(soup.select('p.t-16')[0].get_text().replace('\n','').strip())
        except:
            print("could not find 'salary' on page")
            try:
                salaries.append(re.find('($.)', desc).replace('\n','').strip())
            except:
                print("could not find 'salary' in description")
                try: 
                    salaries.append(soup.select('li.jobs-unified-top-card__job-insight')[0].get_text()
                        .replace('\n','').strip().rsplit(' ', 1)[0].rstrip(' ·'))
                except:
                    print("could not find 'salary' in full-time")
                    salaries.append(None)

        search.append(key)

    # pass data from lists into a dictionary
    dict_from_scrape = {'search':search, 'title':titles, 'company':companies, 'location':locations, 'post_date':post_dates, 
                        'num_applicants':num_applicants, 'full_time':full_time, 'size':size, 
                        'easy_apply':easy_apply, 'description':desc, 'salary':salaries}

    df_from_scrape = pd.DataFrame(dict_from_scrape)

    return df_from_scrape


In [8]:
job_links = {'data analyst':[], 'data scientist':[]} # capture all links to scrape with mathing search term. Titles taken from previous market analysis # FIX: add all roles
df = pd.DataFrame() # init dataframe to capture all jobs

# for each role title given above, return a list of up to 1,000 jobs from LinkedIn
for key, value in job_links.items():
    new_list = [scrape_links(key.replace(' ','%20'))]
    value.append(new_list)
print(job_links) # FIX: debugging

# FIX: check for dupliate links

# Scrape each new link
df = pd.concat([df, scrape_listing(job_links)], axis=0).reset_index(drop=True)
driver.close()

df

running search for 1 of 40 pages for data%20analyst.
length of job links:  7
running search for 2 of 40 pages for data%20analyst.
length of job links:  7
running search for 1 of 40 pages for data%20scientist.
length of job links:  7
running search for 2 of 40 pages for data%20scientist.
length of job links:  7
{'data analyst': [['/jobs/view/3216076186/?eBP=CwEAAAGC1cAUuRdvomRZiZM3b0_RgXd7rsIErLbReCptjr0I9mHwDnJ7gK4-UjRwKf4QQ0em7vTvHkZrGOXGByszt1zQ9bP0Uahd3tSfU54meROaJ46f9vUZshNYYMvy9ohdy6OaNGnNm2u7BdsWUcVHda2oHcqQxydqVTGWZ8cgwPrISvUMcPuy1olLuYxhcTD9twgJfd2PG2yirgwcm-59-tagEyW8zyBRTM6yCax2WjWvqKa1ye0RC1YG65d31F-dvYIv7Ex30ICKF6AS8a5Wro-vHMvyh_nRTL6lM3XpbP_n8hoFpplhw3cutnKqz5569MRmnDSjc10qbl2boT46frDQYCegTtrlpCSiuLRWdAMNorVMkuo9x7tSRrfZX3g&recommendedFlavor=ACTIVELY_HIRING_COMPANY&refId=Y%2Fne8MG7cAs2JxJC83ZS0g%3D%3D&trackingId=%2FnODt1yXzGrSrxR3n37zGQ%3D%3D&trk=flagship3_search_srp_jobs']], 'data scientist': [['/jobs/view/3141978348/?eBP=CwEAAAGC1cBIMpxhbEhinD9QEiqX4m-2zOyxtZ7gBzEEd39din

IndexError: list index out of range

## Scrape Glassdoor using the Selenium Driver

In [ ]:
# search glassdoor for all job titles
# save to glass_df: job title, salary range (or salary avg? both?)
# add glassdoor_salary column to df
# fill glassdoor_salary based on role name

## EDA and Cleaning

In [ ]:
"""
# replace null values
df.isnull().sum()
df.dropna(inplace=True)

# given previous data sets, check salary info
df.salary.unique()

# remove duplicates
clean.rm_dups(df)

df
"""

## Export Data to CSV for Visualization

In [ ]:
# export df to csv

## Challenges & Lessons

### Text extraction & different libraries: 
I couldn't extract text from my scraped LinkedIn data because I was try to pass data from one library's format (Selenium) into another library (Beautiful Soup). I restarted my kernel, rewrote my code (a lot), and one solution I found online used a function similar to others I had found. This seems to transfer text into a different format so that it's readable by other libraries.